In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import urllib.request
import json
import glob
import sys
import os
import hashlib
import hmac
import base64
import random

from tqdm.notebook import tqdm
from datetime import datetime
from pandas.io.json import json_normalize
from prophet import Prophet
from dateutil.relativedelta import *

import warnings
warnings.filterwarnings(action='ignore')

# 한글 글씨체 설정
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.grid'] = False

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)
#소수점 아래 5자리까지 표시
pd.options.display.float_format = '{: .5f}'.format

# 네이버 API 함수 (API키 숨기기 추가 필요)
- https://wooiljeong.github.io/python/naver_datalab_open_api/

# API 인증 정보 설정(yaml)
import yaml
with open('./config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
# 검색어 선택
from config import *
_cfg = config

# 자체 검색어
keyword_list = _cfg['Keyword']['User_list']
# 식신 카테고리
keyword_list = _cfg['Keyword']['Siksin']
# 식품첨가물용어집(식품안전나라 용어사전)
keyword_list = _cfg['Keyword']['Food_Safety_Dict']
# 네이버 식품 카테고리 인기검색어 top500
keyword_list = _cfg['Keyword']['Naver_Food_Rank']
# 요식업 브랜드명
keyword_list = _cfg['Keyword']['Food_Brand']
# 식품안전정보 검색어(식품안전정보원 글로벌 정보부)
keyword_list = _cfg['Keyword']['Food_Safety_Issue']
# 식품안전나라 검색어 기록(22/09/13 기준)
keyword_list = _cfg['Keyword']['Food_Safety_Search']


# 검색어 통합
keyword_list = sum(_cfg['Keyword'].values(), [])
# 중복제거
keyword_unique = []
[keyword_unique.append(x) for x in keyword_list if x not in keyword_unique].clear() #None 반복 출력을 막기 위한 clear

#특수문자 제거
keyword_list = sum([re.findall('[가-힣a-z0-9]+', x) for x in keyword_unique], [])

In [ ]:
# 식품안전나라 용어사전 원본파일
#df_dict = pd.read_csv('src/dict_json.csv') 
#dict_list = list(df_dict['단어'])
#keyword_list = dict_list
#필터링 이후 파일(find_error_list() 함수 실행 후 저장) -653/813
df_dict = pd.read_csv('data/input_keyword/Food_info_dict.csv', encoding='cp949', header=None)
keyword_list = list(df_dict.T[0])



# 네이버 식품카테고리 인기검색어 500
#방법1
with open('data/input_keyword/1015_1115_식품검색어500.txt','r',encoding='utf-8') as f:
    a = f.readlines()
#특수문자 제거
keyword_list = sum([re.findall('[가-힣a-z0-9]+', x) for x in a], [])
#방법2
keyword_list = list(pd.read_table('data/input_keyword/1015_1115_식품검색어500.txt', header=None)[0])

In [127]:
class Parameter():
    def __init__(self):
        self.num = random.randrange(4)
        # 검색어 지정
        #keyword = keyword_list##################
        """
        # API 인증 정보 설정(yaml)
        import yaml
        with open('./config.yaml', encoding='UTF-8') as f:
            _cfg = yaml.load(f, Loader=yaml.FullLoader)
        """
        self.client_id = _cfg['NAVER_TREND'][f'client_id{self.num}']
        self.client_secret = _cfg['NAVER_TREND'][f'client_secret{self.num}']
    # 광고API 인증정보
    BASE_URL = _cfg['NAVER_AD']['BASE_URL']
    API_KEY = _cfg['NAVER_AD']['API_KEY']
    SECRET_KEY = _cfg['NAVER_AD']['SECRET_KEY']
    CUSTOMER_ID = _cfg['NAVER_AD']['CUSTOMER_ID']
    
    # 요청 파라미터 설정
    """
    요청 결과 반환
    timeUnit - 'date', 'week', 'month'
    device - None, 'pc', 'mo'
    ages = [], ['1' ~ '11']
    gender = None, 'm', 'f'
    """
    startDate = str(datetime(2016, 1, 1).strftime('%Y-%m-%d')) #네이버API 최초 제공일자
    endDate = str(datetime.now().date().strftime('%Y-%m-%d')) #오늘
    timeUnit = 'date'
    device = ''
    ages = []
    gender = ''

IndentationError: unexpected indent (1662004970.py, line 7)

In [135]:
class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """
    
    def __init__(self):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        parameter_ = Parameter()
        self.client_id = parameter_.client_id
        self.client_secret = parameter_.client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
        #print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self):
        # Request body
        body = json.dumps({
            "startDate": Parameter.startDate,
            "endDate": Parameter.endDate,
            "timeUnit": Parameter.timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": Parameter.device,
            "ages": Parameter.ages,
            "gender": Parameter.gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            df = pd.DataFrame(result['results'][0]['data'])[['period']]
            for i in range(len(self.keywordGroups)):
                tmp = pd.DataFrame(result['results'][i]['data'])
                tmp = tmp.rename(columns={'ratio': result['results'][i]['title']})
                df = pd.merge(df, tmp, how='left', on=['period'])
            self.df = df.rename(columns={'period': '날짜'})
            self.df['날짜'] = pd.to_datetime(self.df['날짜'])
            
        else:
            print("Error Code:" + rescode)
            
        return self.df

In [136]:
# 한번 실행
# 5개 이하 검색어만 허용
def one_search(keyword):
    if type(keyword) == str:
        naver = NaverDataLabOpenAPI()
        naver.add_keyword_groups({'groupName': keyword, 'keywords': [keyword]})
    elif type(keyword) == list: # 2개 이상 검색어 입력시 리스트로
        naver = NaverDataLabOpenAPI()
        for i in keyword: # 최대5개
            naver.add_keyword_groups({'groupName': i, 'keywords': [i]})
    df = naver.get_data()
    return df

In [137]:
df = one_search('식품안전나라')
df

,날짜,식품안전나라
0,2016-10-25,0.00100
1,2016-12-09,0.00140
2,2017-01-23,0.04181
3,2017-01-24,0.03240
4,2017-01-25,0.03881
...,...,...
2203,2023-02-02,0.59137
2204,2023-02-03,0.70961
2205,2023-02-04,0.11103
2206,2023-02-05,0.10343


# 네이버스토어 순위

In [94]:
def store_rank(keyword):
    # API 인증 정보 설정
    client_id = Parameter.client_id
    client_secret = Parameter.client_secret
    
    encText = urllib.parse.quote(keyword) 
    shop_url = "https://openapi.naver.com/v1/search/shop?query=" + encText + "&amp;display=100&amp;start=1" #100순위
    request = urllib.request.Request(shop_url) 
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        json_str = response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)
    json_object  = json.loads(json_str) #json 변환
    #json_object
    #df = pd.DataFrame(json_object['items'])

    #제목, 최저가, 몰이름, 브랜드, 대분류, 중분류, 소분류, 세부
    #df.loc[ :, ['title', 'lprice', 'mallName', 'brand', 'productId','category1', 'category2', 'category3', 'category4']] 

    # 쇼핑 검색결과
    n = 1
    for i in range(0, len(json_object['items'])) :
        title = json_object['items'][i]['title']
        title = title.replace('&lt;b&gt;', "")
        title = title.replace('&lt;/b&gt;', "")
        mallName = json_object['items'][i]['mallName']
        print(str(n) + ',' + title +',' + mallName)
        n += 1

In [96]:
#1번 키로만 가능
store_rank('노트북')

AttributeError: type object 'Parameter' has no attribute 'client_id'

# 시각화 함수

In [ ]:
def plot_daily_trend(df):
    """
    일 별 검색어 트렌드 그래프 출력
    """
    colList = df.columns[1:]
    n_col = len(colList)

    fig = plt.figure(figsize=(12,6))
    plt.title('일 별 검색어 트렌드', size=20, weight='bold')
    for i in range(n_col):
        sns.lineplot(x=df['날짜'], y=df[colList[i]], label=colList[i])
    plt.legend(loc='upper right')

    return fig

def plot_monthly_trend(df):
    """
    월 별 검색어 트렌드 그래프 출력
    """
    df = df.copy()
    df_0 = df.groupby(by=[df['날짜'].dt.year, df['날짜'].dt.month]).mean().droplevel(0).reset_index().rename(columns={'날짜': '월'})
    df_1 = df.groupby(by=[df['날짜'].dt.year, df['날짜'].dt.month]).mean().droplevel(1).reset_index().rename(columns={'날짜': '년도'})

    df = pd.merge(df_1[['년도']], df_0, how='left', left_index=True, right_index=True)
    df['날짜'] = pd.to_datetime(df[['년도','월']].assign(일=1).rename(columns={"년도": "year", "월":'month','일':'day'}))

    colList = df.columns.drop(['날짜','년도','월'])
    n_col = len(colList)

    fig = plt.figure(figsize=(12,6))
    plt.title('월 별 검색어 트렌드', size=20, weight='bold')
    for i in range(n_col):
        sns.lineplot(x=df['날짜'], y=df[colList[i]], label=colList[i])
    plt.legend(loc='upper right')

    return fig

def plot_pred_trend(df, days): #Prophet 예측 모델
    """
    검색어 시계열 트렌드 예측 그래프 출력
    days: 예측일수
    """
    colList = df.columns[1:]
    n_col = len(colList)

    fig_list = []
    for i in range(n_col):

        globals()[f"df_{str(i)}"] = df[['날짜', f'{colList[i]}']]
        globals()[f"df_{str(i)}"] = globals()[f"df_{str(i)}"].rename(columns={'날짜': 'ds', f'{colList[i]}': 'y'})

        m = Prophet()
        m.fit(globals()[f"df_{str(i)}"])

        future = m.make_future_dataframe(periods=days)
        forecast = m.predict(future)
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

        globals()[f"fig_{str(i)}"] = m.plot(forecast, figsize=(12,6))
        plt.title(colList[i], size=20, weight='bold')

        fig_list.append(globals()[f"fig_{str(i)}"])

    return fig_list

# 최대값 찾기

In [ ]:
# 비율 값이 가장 작은 검색어 찾기 (사용은 하지않음)
def min_key(data):
    min_value = data.min()[1:].values.min()
    data_ = data[data.values == min_value]
    for i in data_.columns:
        try:
            if (data_[i].values == min_value):
                return i
        # 최소값이 같은 검색어가 있을 경우 오류 발생
        except:
            if (data_[i].values[:1] == min_value):
                return i
            
# 비율 값이 가장 큰(100) 검색어 찾기 
def max_key(data):
    data_ = data[data.values == 100]
    for i in data_.columns:
        try:
            if (data_[i].values == 100):
                return i
        except:
            if (data_[i].values[:1] == 100):
                return i

In [ ]:
# 기준열인 100값 찾기 (오류가 발생한다면, 모든 값에 대해 검색값을 갖는 검색어가 없다는 의미)
def find_max_key(keyword): #input 형태는 반드시 리스트
    # 리스트에 중복값 있을 경우 삭제 (순서가 바뀔 수 있지만, 시간복잡도를 고려해 max값만 찾는 경우에는 set연산 사용)
    keyword = [x.replace(' ','') for x in keyword] #공백제거(한달치 검색량에 안 잡힘)
    my_set = set(keyword) #집합set으로 변환
    keyword_unique = list(my_set) #list로 변환
    keyword = keyword_unique.copy()
    
    #검색결과가 너무 적은 경우는 오류처리 하는 것으로 보임
    error_list = []
    
    #결측치가 있는 경우 다른 검색어에 대해서도 결측값이 생기는 점 확인. 이를 대비하기 위한 결측치 리스트 생성
    single_list = []
    
        
    #한번 미리 실행
    ############################################ 처음 검색하는 단어,제외되고 처음이 된 단어가 검색결과가 없는 데이터면 오류발생
    while True:
        try:
            df = one_search(keyword[0])
            break
        except:
            error_list.append(keyword[0])
            keyword.remove(keyword[0])
            
    while len(df) != (datetime.now() - datetime(2016, 1, 1)).days: #첫 검색어는 모든 날짜에 값이 존재하는 검색어가 와야 함
        single_list.append(keyword[0])
        keyword.remove(keyword[0])
        while True:
            try:
                df = one_search(keyword[0])
                break
            except:
                error_list.append(keyword[0])
                keyword.remove(keyword[0])
        
    # 검색어 리스트가 4의 배수가 아니면 4의 배수로 만들기 (검색오류 대비)
    while (len(keyword)%4-1) != 0:
        keyword.append('임시')
        
    df_total = df.copy()

    # 검색어 리스트 값을 키워드로 지정
    for i in tqdm(range(1, len(keyword)-3, 4)):
        key1 = keyword[i]
        key2 = keyword[i+1]
        key3 = keyword[i+2]
        key4 = keyword[i+3]

        max_key_ = max_key(df) # 최대값 비교를 위한 복사
        
        # 데이터 프레임 정의 (one_search([max_key_,key1,key2,key3,key4]로 바꾸면 안 되나??)
        keyword_group_set = {
            'keyword_group_1': {'groupName': max_key_, 'keywords': [max_key_]},
            'keyword_group_2': {'groupName': key1, 'keywords': [key1]},
            'keyword_group_3': {'groupName': key2, 'keywords': [key2]},
            'keyword_group_4': {'groupName': key3, 'keywords': [key3]},
            'keyword_group_5': {'groupName': key4, 'keywords': [key4]}
        }
        
        naver = NaverDataLabOpenAPI()

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_2'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_3'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_4'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_5'])
        
        try:
            df = naver.get_data()

            # 결측치 있는 최대값후보 single_list에 넣기
            while df[max_key(df)].isnull().any(): #결측값이 하나라도 있으면 max_key가 되면 안 된다.
                single_list.append(max_key(df))
                temp_list = [key1, key2, key3, key4]
                temp_list.remove(max_key(df))
                while (len(temp_list) != 3):
                    temp_list.append('임시')

                naver = NaverDataLabOpenAPI()
                naver.add_keyword_groups({'groupName': max_key_, 'keywords': [max_key_]})
                naver.add_keyword_groups({'groupName': temp_list[0], 'keywords': [temp_list[0]]})
                naver.add_keyword_groups({'groupName': temp_list[1], 'keywords': [temp_list[1]]})
                naver.add_keyword_groups({'groupName': temp_list[2], 'keywords': [temp_list[2]]})
                df = naver.get_data()

                
                # 이 부분 꼭 필요한지 모르겠음 실험해보기~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            if max_key_ == max_key(df):
                df_total = pd.merge(df_total,df, how='left', on=["날짜",max_key(df)])
            if max_key_ != max_key(df):
                df_total = df_total.drop([max_key_],axis=1)
                df_total = pd.merge(df_total,df, how='left', on="날짜")
                
        except:
            error_list.append([key1,key2,key3,key4])
    
    if len(single_list) > 0 : #single_list에 total_max_key를 추가하는 이유는, total_max_key와 비교하기 위함
        single_list.insert(0, max_key(df_total))
    return max_key(df_total), single_list, error_list #single_list는 결측값이 있는 검색어 중 최대값일 가능성이 있는 단어추후 업데이트(개별로 one_search로 구해서 한달치에 곱하고 merge 하는게 좋아보임)

In [ ]:
total_max_key, single_list, error_list = find_max_key(keyword_list)

In [ ]:
total_max_key # 결측치가 없는 최대값

In [ ]:
single_list

In [ ]:
error_list

### error_list 찾기

In [ ]:
error_list = sum(error_list, [])
error_list

In [ ]:
def find_error_list(error_list, total_max_key, single_list):
    final_error = []
    for i in error_list:
        try:
            df = one_search([total_max_key, i])
            # 결측치 있는 최대값후보 single_list에 넣기
            if total_max_key != max_key(df):
                if df[max_key(df)].isnull().any(): #결측값이 하나라도 있으면 max_key가 되면 안 된다.
                    single_list.append(max_key(df))
                else:
                    total_max_key = max_key(df)
        except:
            final_error.append(i)
    return total_max_key, single_list, final_error

In [ ]:
total_max_key, single_list, final_error = find_error_list(error_list, total_max_key, single_list)

In [ ]:
total_max_key

In [ ]:
final_error #검색량 검색결과 하루의 결과도 반환하지 않는 경우

In [ ]:
# 문제 없는 키워드              ######################################singlr_list 뒤로 안 가도 되나?
keyword_list = [x for x in keyword_list if x not in final_error]
keyword_list = [x.replace(' ','') for x in keyword_list] #공백제거(한달치 검색량에 안 잡힘)
len(keyword_list)

In [ ]:
import csv
# 저장
with open('data/input_keyword/total_keyword.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(keyword_list)

In [ ]:
# Foodlist를 바꾸지 않았으면 여기부터!!

In [ ]:
# 불러오기
import csv
keyword_list = []
with open('data/input_keyword/total_keyword.csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        keyword_list.append(row)
keyword_list = sum(keyword_list, [])

In [ ]:
len(keyword_list)

### single_list 중 total_max_key보다 작을 수도 있는 값 추가

In [ ]:
#결측치 있던 max_key()후보와 total_max_key를 다시 비교해서 최종 total_max_key보다 작은 값은 포함
def find_unmax_key(keyword):
    if len(keyword) == 0:
        print("모든 검색어가 이미 포함되었습니다.")
        return []
    
    elif len(keyword) > 0:
        single_list = []

        # 검색어 리스트가 4의 배수가 아니면 4의 배수로 만들기 (검색오류 대비)
        while (len(keyword)%4-1) != 0:
            keyword.append('임시')

        #한번 미리 실행
        df = one_search(keyword[0])
        df_total = df.copy()

        # 검색어 리스트 값을 키워드로 지정
        for i in range(1, len(keyword)-3, 4):
            key1 = keyword[i]
            key2 = keyword[i+1]
            key3 = keyword[i+2]
            key4 = keyword[i+3]

            # 데이터 프레임 정의
            keyword_group_set = {
                'keyword_group_1': {'groupName': keyword[0], 'keywords': [keyword[0]]},
                'keyword_group_2': {'groupName': key1, 'keywords': [key1]},
                'keyword_group_3': {'groupName': key2, 'keywords': [key2]},
                'keyword_group_4': {'groupName': key3, 'keywords': [key3]},
                'keyword_group_5': {'groupName': key4, 'keywords': [key4]}
            }

            naver = NaverDataLabOpenAPI()

            naver.add_keyword_groups(keyword_group_set['keyword_group_1'])
            naver.add_keyword_groups(keyword_group_set['keyword_group_2'])
            naver.add_keyword_groups(keyword_group_set['keyword_group_3'])
            naver.add_keyword_groups(keyword_group_set['keyword_group_4'])
            naver.add_keyword_groups(keyword_group_set['keyword_group_5'])
            df = naver.get_data()

            if keyword[0] != max_key(df):
                for i in [key1,key2,key3,key4]:
                    df_ = df.iloc[:,1:]
                    if df_[keyword[0]].max() < df_[i].max():
                           single_list.append(i)
        print('정상적으로 처리되었습니다.')                  
        return single_list #single_list는 결측값이 있는 검색어 중 최대값일 가능성이 있는 단어추후 업데이트(개별로 one_search로 구해서 한달치에 곱하고 merge 하는게 좋아보임)

In [ ]:
# 최종 제거값
single_list = find_unmax_key(single_list)

In [ ]:
single_list

# 검색량 출력

In [ ]:
keyword_unique = []
for v in keyword_list:
    if v not in keyword_unique:
        keyword_unique.append(v)
keyword = keyword_unique.copy()

In [ ]:
len(keyword)

In [ ]:
def search_amount(keyword):
    keyword = [x for x in keyword if x not in single_list] #single_list 검색어는 제외. 추후 실제값으로 합칠 예정
    keyword = [x.replace(' ','') for x in keyword]
    # 리스트에 중복값 있을 경우 삭제 (시간복잡도가 늘어나더라도 순서가 바뀌지 않도록 for문 사용)
    keyword_unique = []
    for v in keyword:
        if v not in keyword_unique:
            keyword_unique.append(v)
    keyword = keyword_unique.copy()

    # find_max_key를 맨 앞으로 (나중에 완성되면 total_max_key를 max_key(keyword) 로)
    keyword.remove(total_max_key)
    keyword.insert(0,total_max_key)

    # 검색어 리스트가 4의 배수가 아니면 4의 배수로 만들기 (검색오류 대비)
    while (len(keyword)%4-1) != 0:
        keyword.append('임시')

    #한번 미리 실행
    naver = NaverDataLabOpenAPI()
    naver.add_keyword_groups({'groupName': keyword[0], 'keywords': [keyword[0]]})
    df = naver.get_data()
    df_total = df.copy()


    # 검색어 리스트 값을 키워드로 지정
    for i in tqdm(range(1, len(keyword)-3, 4)):
        key1 = keyword[i]
        key2 = keyword[i+1]
        key3 = keyword[i+2]
        key4 = keyword[i+3]

        # 데이터 프레임 정의
        keyword_group_set = {
            'keyword_group_1': {'groupName': keyword[0], 'keywords': [keyword[0]]},
            'keyword_group_2': {'groupName': key1, 'keywords': [key1]},
            'keyword_group_3': {'groupName': key2, 'keywords': [key2]},
            'keyword_group_4': {'groupName': key3, 'keywords': [key3]},
            'keyword_group_5': {'groupName': key4, 'keywords': [key4]}
        }


        naver = NaverDataLabOpenAPI()

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_2'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_3'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_4'])
        naver.add_keyword_groups(keyword_group_set['keyword_group_5'])
        df = naver.get_data()
        
        df_total = pd.merge(df_total,df, how='left', on=["날짜",keyword[0]])

    #중복값/4의 배수를 만들기 위해 생성한 값 삭제
    if keyword[-1] == '임시':
        #df_total = df_total.T.drop_duplicates().T
        df_total = df_total[df_total.columns.drop(list(df_total.filter(regex='임시')))]
        #df.drop(df[df.columns[pd.Series(df.columns).str.startswith('임시')]], axis=1)
    
    #원래 리스트 순서대로 정렬
    keyword_unique.insert(0, '날짜')
    df = df_total[keyword_unique]
    return df

In [ ]:
df = search_amount(keyword_list)

In [ ]:
date = str(datetime.now().date().strftime('%Y%m%d'))
df.to_csv(f'data/result/search_result{date}.csv', index=False)
df = pd.read_csv(f'data/result/search_result{date}.csv')
#df = pd.read_csv('data/news/search_result_naver500_20221116.csv')
df['날짜'] = df['날짜'].astype('datetime64[ns]')
df

In [ ]:
# null이 없는 검색어만 출력
df[df.columns[df.isna().sum() == 0]]

# 특정 검색어의 한달치 실제 검색량 구하기
- https://www.sagein.net/m/652

In [ ]:
from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *
from time import sleep
from urllib.error import HTTPError

In [ ]:
# 검색어의 최근 한달 검색량 반환
def search_keyword(searchword):
    rel = RelKwdStat(Parameter.BASE_URL, Parameter.API_KEY, Parameter.SECRET_KEY, Parameter.CUSTOMER_ID)
    kwdDataList = rel.get_rel_kwd_stat_list(siteId=None, biztpId=None, hintKeywords=searchword, event=None, month=None, showDetail='1')
    kwd_result = (kwdDataList[0].relKeyword, #키워드
                 kwdDataList[0].monthlyPcQcCnt, #월간 검색수 (PC)-최근30일
                 kwdDataList[0].monthlyMobileQcCnt, # 월간 검색수 (Mobile)-최근30일
                 kwdDataList[0].monthlyPcQcCnt+kwdDataList[0].monthlyMobileQcCnt) # 월간 total 

    return(kwd_result[3])
    #[outdata.relKeyword for outdata in kwdDataList] ---> 연관검색어
    #return kwd_result

In [ ]:
# 식품안전나라 최근 1달 검색 절대값
search_keyword('식품안전나라')

In [ ]:
# 식품안전나라 최근 1달 상대값 합계
df['식품안전나라'].tail(30).sum()

# 실제값으로 변환

In [ ]:
def real_amount(keyword):
    #결측값이 있는 검색어가 포함되지 않도록(식품안전나라와 같이 결측값이 측정에 들어가면 오차 수치가 낮아짐. 예외처리 해줘야함)
    keyword = [x.replace(' ','') for x in keyword]
    sample_candidate = []
    for i in df.columns: 
        if (df[i].isnull().any() == False):
            sample_candidate.append(i)
    keyword = [x for x in keyword if x not in single_list] #개별로 계산한 single_list로 계산하지 않도록
    keyword = [x for x in keyword if x in sample_candidate]

    #실제값 변환, 오차율,MSE를 구하기 위한 샘플
    keyword_sample = random.sample(keyword, 5) #5개만으로 추출하지만, 예비용으로 10개 설정

    #최근 한달이 이전30일을 제공하기 떄문에 -1month 가 아닌 -30days(오늘 제외)
    start_date = str(datetime.now().date() - relativedelta(days=30)) #한달 전
    end_date =  str(datetime.now().date() - relativedelta(days=1)) #어제

    calculator_dict = {}
    #last_month_index = int(df[df['날짜']==start_date].index.values)
    month_amount_dict = {}
    for i in keyword_sample:
        last_month = df.tail(30)[i] #원래 인덱스로 읽어왔으나, 결측값이 있는 경우 인덱스가 줄어들기 때문에 tail(30)이 정확하다고 판단하여 수정
        month_amount = search_keyword(i) # 검색어 하나의 한달 실제 검색량
        month_amount_dict[i] = month_amount
        calculator = month_amount / last_month.sum() # 100의 값(100값 x 비율(values) 하면 실제 값이 나옴)
        calculator_dict[i] = calculator
        sleep(0.3)

    calculator = sum(calculator_dict.values()) / len(keyword_sample)
    result = df.iloc[:,1:] * calculator
    final_df = pd.concat([df['날짜'], result], axis=1)

    #성능평가(오차율, MSE). 오차를 측정할 경우 일일 호출량이 더 사용된다.
    print("5개의 샘플을 추출하여 오차율과 MSE를 계산 중 입니다...")
    error_per_list = []
    MSE_list =[]
    for i in calculator_dict.items():
        temp_df = one_search(i[0])
        real_amount_month = month_amount_dict[i[0]] / temp_df[i[0]].tail(30).sum()
        real_amount_df = real_amount_month * temp_df[i[0]].tail(30)
        error_per = (abs(real_amount_df.mean() - result.tail(30)[i[0]].mean())*100/real_amount_df.mean())
        MSE = np.square(real_amount_df.mean() - result.tail(30)[i[0]].mean()).sum()/len(result)
        error_per_list.append(error_per)
        MSE_list.append(MSE)        
        print(f"검색어 '{i[0]}'에 대한 오차율: {error_per}, MSE: {MSE}")
        
    #오차율 기준(10%)을 넘는 샘플이 있으면 다시 샘플 추출
    if (len(keyword) >= 50) and (sorted(error_per_list, reverse=True)[0] >= 10): #리스트 내 검색어 수가 50개가 넘을 시에만 실행
        print('오차율 10% 넘는 샘플이 발견되어 샘플을 다시 추출합니다..')
        real_amount(keyword) #재귀함수

    #결측값이 있으면서 total_max_key보다 큰 값은 따로 concat (이상치여도 어쩔 수 없음)
    if len(single_list) > 0:
        for i in single_list:
            single_df = one_search([total_max_key,i])
            single_amount = search_keyword(i) / single_df[i].tail(30).sum()
            single_amount_df = single_amount * single_df[i]
            final_df = pd.concat([final_df, single_amount_df], axis=1, sort=True)
    return final_df, np.mean(error_per_list), np.mean(MSE_list)

In [ ]:
final_df, error_per_average, MSE_average = real_amount(keyword_list)

In [ ]:
# 오차율(5개 샘플 평균)
error_per_average

In [ ]:
# 평균제곱오차(5개 샘플 평균)
MSE_average

# 시각화
- 아래는 발전 가능성?
- https://cordingdiary.tistory.com/109
- https://www.analyticsvidhya.com/blog/2021/12/anomaly-detection-model-using-facebook-prophet/
- https://predictor-ver1.tistory.com/4
- https://zzsza.github.io/data/2019/02/06/prophet/

In [ ]:
# 시계열 그래프 작성을 위해 값이 2개 최소 이상인 검색어만 추출
df_ = final_df[final_df.columns[final_df.isnull().sum() < len(final_df)-1]]

In [ ]:
fig_1 = plot_monthly_trend(df_)

In [ ]:
fig_2 = plot_daily_trend(df_)

In [ ]:
# NAN값이 많은 데이터는 예측이 떨어짐. 값이 1개밖에 없는 경우 오류발생
fig_3 = plot_pred_trend(df_, days = 365)

# 상관관계 시각화

In [ ]:
# null값이 없는 열만 시각화 (식품안전나라도 제외되므로 포함시 유의)
sample_candidate = []
for i in df_.columns:
    if (df_[i].isnull().any() == False):
        sample_candidate.append(i)
keyword = keyword_list.copy()
#keyword = [x for x in keyword if x not in single_list]
keyword = [x for x in keyword if x in sample_candidate]

df__ = df_[keyword]
df__ = df__[final_df['날짜'] >= '2016-01-01'].reset_index().drop(['index'],axis=1)

In [ ]:
df__.corr()

In [ ]:
# 그림 사이즈 지정
fig, ax = plt.subplots(figsize=(20,20))

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(df__.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(df__.corr(), 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )  
plt.show()

# 정규화해서 곱해 실제값을 구하는 방법(호출량 절약)
- https://foss4g.tistory.com/1410

# 연령대별 검색량(실제값 변환은 어려움)
- https://yenpa.tistory.com/m/10

# 연관검색어
- https://yenpa.tistory.com/m/20

# 거리계산(군집 나누기)

df_ = result_df.max().drop('날짜')

from scipy.spatial import distance
print(distance.euclidean(df_['포도'], df_['사과']))
print(distance.cityblock(df_['포도'], df_['사과']))
print(distance.hamming(df_['포도'], df_['사과']))

df.max() - df.mean()